In [1]:
from bert import BertModel

/home/krypticmouse/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.6.output.adapter.proj_down.bias', 'bert.encoder

In [3]:
from dataset import CoLADataset
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
data = CoLADataset('raw/in_domain_train.tsv', tokenizer)

In [5]:
model(**data[0]['sentence'])

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_32814/3281607745.py:1 in <module>                                          │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_32814/3281607745.py'                 │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1194 in _call_impl     │
│                                                                                           │
│   1191 │   │   # this function, and just call forward.                                    │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_h │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):            │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                  │
│   1195 │   │   # Do not call functions when jit is used                                   │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                      │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                         │
│                                                                                           │
│ /home/krypticmouse/Desktop/Adapter-BERT/bert.py:998 in forward                            │
│                                                                                           │
│    995 │   │   │   inputs_embeds=inputs_embeds,                                           │
│    996 │   │   │   past_key_values_length=past_key_values_length,                         │
│    997 │   │   )                                                                          │
│ ❱  998 │   │   encoder_outputs = self.encoder(                                            │
│    999 │   │   │   embedding_output,                                                      │
│   1000 │   │   │   attention_mask=extended_attention_mask,                                │
│   1001 │   │   │   head_mask=head_mask,                                                   │
│                                                                                           │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1194 in _call_impl     │
│                                                                                           │
│   1191 │   │   # this function, and just call forward.                                    │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_h │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):            │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                  │
│   1195 │   │   # Do not call functions when jit is used                                   │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                      │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                         │
│                                                                                           │
│ /home/krypticmouse/Desktop/Adapter-BERT/bert.py:588 in forward                            │
│                                                                                           │
│    585 │   │   │   │   │   encoder_attention_mask,                                        │
│    586 │   │   │   │   )                                                                  │
│    587 │   │   │   else:                                                                  │
│ ❱  588 │   │   │   │   layer_outputs = layer_module(                                      │
│    589 │   │   │   │   │   hidden_states,                                                 │
│    590 │   │   │   │   │   attention_mask,        

In [10]:
3072/768

4.0